In [3]:
import os
import shutil
import numpy as np
import torch.nn as nn
from torch.autograd import Variable

In [5]:
os.listdir(os.getcwd())

['ros',
 'models',
 '__init__.py',
 'Untitled.ipynb',
 '.ipynb_checkpoints',
 'main.py',
 'utils',
 'data']

In [14]:
t = os.path.join(os.getcwd(), 'results')
print t
if os.path.isdir(t):
    print 'yes'
else:
    print 'no'

/home/lex/catkin_ws/src/soft-neuro-adapt/pyrnn/src/results
no


In [9]:
help(os.path.join)

Help on function join in module posixpath:

join(a, *p)
    Join two or more pathname components, inserting '/' as needed.
    If any component is an absolute path, all previous path components
    will be discarded.  An empty last part will result in a path that
    ends with a separator.



In [20]:
import csv
testF = open(os.path.join(os.getcwd(), 'test_csv'), 'w')
testW = csv.writer(testF)
help(testW.writerow)

Help on built-in function writerow:

writerow(...)
    writerow(sequence)
    
    Construct and write a CSV record from a sequence of fields.  Non-string
    elements will be converted to string.



In [50]:
import math

def toRad(rad):
    return (rad * math.pi)/180.0

def excitation_input():
    x = npr.randint(0, 90, dtype='int64')
    x = toRad(x)
    return np.sin(x)
    
excitation_input()

0.99939082701909576

In [42]:
import numpy.random as npr
# help(npr.randint)

Help on built-in function randint:

randint(...)
    randint(low, high=None, size=None, dtype='l')
    
    Return random integers from `low` (inclusive) to `high` (exclusive).
    
    Return random integers from the "discrete uniform" distribution of
    the specified dtype in the "half-open" interval [`low`, `high`). If
    `high` is None (the default), then results are from [0, `low`).
    
    Parameters
    ----------
    low : int
        Lowest (signed) integer to be drawn from the distribution (unless
        ``high=None``, in which case this parameter is one above the
        *highest* such integer).
    high : int, optional
        If provided, one above the largest (signed) integer to be drawn
        from the distribution (see above for behavior if ``high=None``).
    size : int or tuple of ints, optional
        Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
        ``m * n * k`` samples are drawn.  Default is None, in which case a
        single value is

In [38]:
np.arcsin(0)

0.0